In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats


Загрузка данных

In [0]:
def read_data(file, col_name):
    df = pd.read_csv(file)
    df.columns = ["date", col_name]
    df["date"] = pd.to_datetime(df["date"], dayfirst=True)
    df.sort_values(by="date", inplace=True)
    return df

df = read_data("data/ruonia.csv", "r_hm")
df = df.merge(read_data("data/LIBOR USD.csv", "r_fr"), on="date", how="outer")
df = df.merge(read_data("data/usd_exch_rate.csv", "fx"), on="date", how="outer")
df.dropna(inplace=True)
df["fx"] = 1/df["fx"]
df["r_hm"] = df["r_hm"]/100
df["r_fr"] = df["r_fr"]/100
df.head()

Оценка матрицы ковариации и разложение Холецкого

In [0]:
from scipy.linalg import cholesky

cov_mat = np.concatenate([
    np.diff(df["r_hm"])[:, None],
    np.diff(df["r_fr"])[:, None],
    (np.diff(df["fx"])/np.array(df["fx"])[:-1])[:, None]
], axis=1).T

cov_mat = np.cov(cov_mat)
chol = cholesky(cov_mat, lower=True)
chol

Симуляция скоррелированных Винеровских процессов

In [0]:
steps = 252*2
n_sim = 100
dt = 1/252

dw = np.random.normal(size=(3,n_sim,steps))
dw = np.tensordot(chol, dw, axes=1)

Симуляция ставок и обменного курса

In [0]:
def gen_Vasicek(dw, a, b, sigma, dt, steps, r0, n_sim):
    r = np.zeros(shape=(n_sim, steps+1))
    r[:,0] = r0
    for i in range(1, steps+1):
        r[:,i] = a*b*dt +(1-a*dt)*r[:,i-1] + sigma*dw[:,i-1]
    return r[:,1:]

def fx_simulate(dw, fx0, r_fr_sim, r_hm_sim, drift, sigma, dt, n_sim, steps):
    
    assert r_fr_sim.shape == (n_sim, steps)
    assert r_hm_sim.shape == (n_sim, steps)
    
    dfx_fx = np.zeros(shape=(n_sim, steps+1))
    dfx_fx[:,0] = fx0
    
    for i in range(1, steps+1):
        dfx_fx[:,i] = (r_hm_sim[:,i-1] - r_fr_sim[:,i-1] + drift)*dt + sigma*dw[:,i-1] + 1
    fx_sim = np.cumprod(dfx_fx, axis=1)
    
    return fx_sim[:,:]

In [0]:
fx_start = 1/69.4706 # на 30.12.2018
dt=1/252
r_hm_params = {"a":0.3, "b":0.3/10, "sigma":0.03, "dt":dt, "steps":steps,
               "r0":df["r_hm"].tolist()[-1], "n_sim":n_sim}
r_fr_params = {"a":0.063, "b":0.03/10, "sigma":0.0093, "dt":dt, "steps":steps,
               "r0":df["r_fr"].tolist()[-1], "n_sim":n_sim}
fx_params = {"drift":0.015, "sigma":0.11, "dt":dt, "steps":steps, "fx0":fx_start, "n_sim":n_sim}

In [0]:
r_hm_sim = gen_Vasicek(dw=dw[1,:,:], **r_hm_params)
r_fr_sim = gen_Vasicek(dw=dw[2,:,:], **r_fr_params)
r_hm_sim.shape

In [0]:
df["r_hm"].mean(), df["r_fr"].mean()

In [0]:
fx_sim = fx_simulate(dw=dw[2,:,:], r_hm_sim=r_hm_sim, r_fr_sim=r_fr_sim, **fx_params)

Визуализация симуляций

In [0]:
pd.DataFrame(r_hm_sim.T).plot()

In [0]:
pd.DataFrame(r_fr_sim.T).plot()

In [0]:
pd.DataFrame(data=fx_sim.T).plot()

Симулирование цен на каждый день

In [0]:
def gkm_value(S0, rd, rf, T, K=0.014, sigma=0.04, regime='put'):
      ''' Valuation of European put/call option in Garman-Kohlhagen model.
      Analytical formula.
      Parameters
      ==========
      S0 : float
      initial stock/index level
      K : float
      strike price
      T : float
      maturity date (in year fractions)
      rd : float
      domestic risk-free short rate
      rf : float
      foreign risk-free short rate
      sigma : float
      volatility factor
      Returns
      =======
      value : float
      present value of the European FX option
      '''
    
      d1 = (np.log(S0 / K) + (rd - rf + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
      d2 = d1 - sigma * np.sqrt(T)
      if regime=='put':
        value = (K * np.exp(-rd * T)* stats.norm.cdf(-d2, 0.0, 1.0) - S0 * np.exp(-rf * T) * stats.norm.cdf(-d1, 0.0, 1.0))
        return value
      elif regime=='call':
        value = (S0 * np.exp(-rf * T) * stats.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-rd * T)* stats.norm.cdf(d2, 0.0, 1.0))
        return value
      else:
        return None             


In [0]:
def forward_value(S0, rd, rf, T, K=0.015, N=10**5):
    """ Форвардная цена, умноженная на номинал 
    rf : Безрисковая процентная ставка по иностранной валюте ( LIBOR)
    rd : Безрисковая процентная ставка по домашней валюте ( RUONIA)
    T : Количество лет 
    S0 :  Текущая цена (спот)"""
    value = N * (S0 * np.exp((rd - rf) * T) - K) 
    
    return value  

In [0]:
# здесь будет своп
### YOUR CODE HERE ###

In [0]:
# здесь будет кривая бескупонной
### YOUR CODE HERE ###

# предположил, что называется r_zero
# размерность (дата ставки(steps), на какой срок(steps))

In [0]:
def discount(St, r, t):
    """ Дисконтирование стоимости актива
    r : годовая ставка дисконтирования
    t : количество лет
    """
#     value = St / (1 + r)**t
    value = St * np.exp(-r * t)
    
    return value

Считаем цены, дисконтируем

In [0]:
IR_swap = np.zeros((steps, n_sim))
FX_fwd = np.zeros((steps, n_sim))
FX_opt = np.zeros((steps, n_sim))

IR_swap_disc = np.zeros((steps, n_sim))
FX_fwd_disc = np.zeros((steps, n_sim))
FX_opt_disc = np.zeros((steps, n_sim))

for step in range(steps):
    IR_swap[step] = ### YOUR CODE HERE ###
    FX_fwd[step] = forward_value(fx_sim.T[step], r_hm_sim[step], r_fr_sim[step], 1-step/steps)
    FX_opt[step] = gkm_value(fx_sim.T[step], r_hm_sim[step], r_fr_sim[step], 1-step/steps)
    
    IR_swap_disc[step] = discount(IR_swap[step], r_zero[step, steps-step], steps-step)
    FX_fwd_disc[step] = discount(FX_fwd[step], r_zero[step, steps-step], steps-step)
    FX_opt_disc[step] = discount(FX_opt[step], r_zero[step, steps-step], steps-step)

Считаем квантили, находим наибольший